In [ ]:
# !pip install openpyxl
# !pip install imbalanced-learn
# !pip3 install ipympl
# !pip install import-ipynb
# !pip install shapely
# !pip install SciencePlots 
# !pip install seaborn
# !pip install tqdm
# !pip install ipywidgets

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots
import os, sys
from numpy import nan
import re
import ipympl
# from IPython.core.display import display, HTML
import ipywidgets
import json
from os import listdir
import glob
import math
from IPython.display import Image, display, HTML
from shapely.geometry import mapping
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split, RepeatedKFold, cross_val_predict
from sklearn.metrics import r2_score, mean_squared_error, precision_score, recall_score, mean_absolute_error, make_scorer
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, LeaveOneOut, cross_validate
from sklearn.impute import SimpleImputer
from tqdm.notebook import tqdm
from sklearn.svm import SVC
from tqdm import tqdm
import pickle
from imblearn.over_sampling import SMOTE
from scipy.stats import ks_2samp
from scipy import stats
np.seterr(divide='ignore', invalid='ignore')
pd.options.display.max_columns = 100
from sklearn.preprocessing import PolynomialFeatures
%matplotlib inline

In [2]:
import import_ipynb
from SoilPrep import * 

importing Jupyter notebook from SoilPrep.ipynb


# Step 1: Data Preprocessing (Normalisation)

In [3]:
# Colour scheme definition
kado = '#8B7355'
mati = '#A52A2A'
balu = '#F4A460'

udf = pd.read_csv('uae.csv')
# print(udf.shape)
# print(udf["FID"].min(), udf["FID"].max())
# Note that some ids are missing in the data

In [4]:
udf['sand'] = lognormal (udf['sand'].copy())
udf['silt'] = lognormal (udf['silt'].copy())
udf['clay'] = lognormal (udf['clay'].copy())
udf['TOC'] = lognormal (udf['TOC'].copy())

# plt.hist(udf['clay'], bins=98)
# plt.show()


## Correlation between wavelengths and Targets

In [5]:
plt.style.use(['science','notebook','grid'])
# Pearson corelation between different wavelengths and Targets/Outputs (i.e, sand, clay, silt, and TOC) 

# First and second row of sand_rp will store r values and p values for different wavelengths
# Similar is the case for silt_rp, clay_rp, and toc_rp
sand_rp = udf.loc[0:1,:].copy()
silt_rp = udf.loc[0:1,:].copy()
clay_rp = udf.loc[0:1,:].copy()
toc_rp = udf.loc[0:1,:].copy()

# Calculating r value and p value for different wavelengths when calculated against different Targets 
for i in range(7,2158):
    sand_rp.iloc[0, i], sand_rp.iloc[1, i] = stats.pearsonr(udf.loc[:,'sand'], udf.iloc[:, i])
    silt_rp.iloc[0, i], silt_rp.iloc[1, i] = stats.pearsonr(udf.loc[:,'silt'], udf.iloc[:, i])
    clay_rp.iloc[0, i], clay_rp.iloc[1, i] = stats.pearsonr(udf.loc[:,'clay'], udf.iloc[:, i])
    toc_rp.iloc[0, i], toc_rp.iloc[1, i] = stats.pearsonr(udf.loc[:,'TOC'], udf.iloc[:, i])
    
# Ploting the pearson correlation coefficient for different wavelengths against different Targets

def plot_corr (target):
    if target == 'sand':
        sand_rp.iloc[0,7:].plot(color = balu)
    elif target == 'silt':
        silt_rp.iloc[0,7:].plot(color = kado) 
    elif target == 'TOC':
        toc_rp.iloc[0,7:].plot(color = 'green')
    elif target == 'clay':
        clay_rp.iloc[0,7:].plot(color = mati)   
    else:
        clay_rp.iloc[0,7:].plot(color = mati)
        sand_rp.iloc[0,7:].plot(color = balu)
        silt_rp.iloc[0,7:].plot(color = kado) 
        toc_rp.iloc[0,7:].plot(color = 'green') 
    plt.ylim([-0.6, 0.6])

ipywidgets.interact(plot_corr, target= ['sand', 'silt', 'clay', 'TOC', 'All'])


interactive(children=(Dropdown(description='target', options=('sand', 'silt', 'clay', 'TOC', 'All'), value='sa…

<function __main__.plot_corr(target)>

## Visualizing Spectrum by Increasing Target Contents (Interactive)

In [14]:
# Sorting spectra by Sand content

smt =25

bySand = udf.sort_values(by =['sand'])
bySand = bySand.reset_index(drop=True)
bySandS = bySand.iloc[:,:].copy()
bySandS.iloc[:, 7:2158] = sgsmooth (bySand.iloc[:, 7:2158],smt)

# Sorting spectra by Silt content
bySilt = udf.sort_values(by =['silt'])
bySilt = bySilt.reset_index(drop=True)
bySiltS = bySilt.iloc[:,:].copy()
bySiltS.iloc[:, 7:2158] = sgsmooth (bySilt.iloc[:, 7:2158],smt)
#bySiltS = sgsmooth (bySilt,25)

# Sorting spectra by Clay content
byClay = udf.sort_values(by =['clay'])
byClay = byClay.reset_index(drop=True)
byClayS = byClay.iloc[:,:].copy()
byClayS.iloc[:, 7:2158] = sgsmooth (byClay.iloc[:, 7:2158],smt)
#byClayS = sgsmooth (byClay,25)

# Sorting spectra by Total Organic Content(TOC)
byTOC = udf.sort_values(by =['TOC'])
byTOC = byTOC.reset_index(drop=True)
byTOCS = byTOC.iloc[:,:].copy()
byTOCS.iloc[:, 7:2158] = sgsmooth (byTOC.iloc[:, 7:2158],smt)
#byTOCS = sgsmooth(byTOC, 25)
# Using ipywidgets to draw interactive plots

In [15]:
def plot_spec (sort_by, sample, smooth):
    if sort_by == 'sand':
        x= bySand.iloc[sample, 7:2158]
        if smooth == 'on':
            x = bySandS.iloc[sample, 7:2158] 
    elif sort_by == 'silt':
        x= bySilt.iloc[sample, 7:2158]
        if smooth == 'on':
            x= bySiltS.iloc[sample, 7:2158]
    elif sort_by == 'TOC':
        x= byTOC.iloc[sample, 7:2158]
        if smooth == 'on':
            x= byTOCS.iloc[sample, 7:2158]
    else:
        x= byClay.iloc[sample, 7:2158]
        if smooth == 'on':
            x = byClayS.iloc[sample, 7:2158]
    x.plot()
    plt.ylim([-0.6, 0.8])

ipywidgets.interact(plot_spec, sort_by= ['sand', 'clay', 'silt', 'TOC'], sample = (0, 293,1), smooth = ['on', 'off'])

interactive(children=(Dropdown(description='sort_by', options=('sand', 'clay', 'silt', 'TOC'), value='sand'), …

<function __main__.plot_spec(sort_by, sample, smooth)>

# Step 3:  Train-Test Split  and its Goodness

In [ ]:
y_silt = udf.loc[:, 'silt']
y_sand = udf.loc[:, 'sand']
y_clay = udf.loc[:, 'clay']
y_toc = udf.loc[:, 'TOC']

In [ ]:
tst_siz = 0.2
X = udf.iloc[:, 7:2158].copy()

rand_sand, min_err = best_split(X, y_sand, tst_siz)
print ('Sand: ')
print ('For test size =', tst_siz, '\t min bin error:', min_err, '\t found at randome state =', rand_sand)

rand_silt, min_err = best_split(X, y_silt, tst_siz)
print ('Silt: ')
print ('For test size =', tst_siz, '\t min bin error:', min_err, '\t found at randome state =', rand_silt)

rand_clay, min_err = best_split(X, y_clay, tst_siz)
print ('Clay: ')
print ('For test size =', tst_siz, '\t min bin error:', min_err, '\t found at randome state =', rand_clay)

rand_toc, min_err = best_split(X, y_toc, tst_siz)
print ('TOC: ')
print ('For test size =', tst_siz, '\t min bin error:', min_err, '\t found at randome state =', rand_toc)

##  Optimal Train-Test Split

In [ ]:
# Splitting the sample into train-test samples using obtained optimal parameters
XSand_train, XSand_test, ySand_train, ySand_test = train_test_split(X, y_sand, test_size= tst_siz, random_state=rand_sand)

XSilt_train, XSilt_test, ySilt_train, ySilt_test = train_test_split(X, y_silt, test_size= tst_siz, random_state=rand_silt)

XClay_train, XClay_test, yClay_train, yClay_test = train_test_split(X, y_clay, test_size= tst_siz, random_state=rand_clay)

XTOC_train, XTOC_test, yTOC_train, yTOC_test = train_test_split(X, y_toc, test_size= tst_siz, random_state=rand_toc)
print ('\n \n Optimal Train-Test Split Done for y_sand y_silt y_clay y_toc')

In [ ]:
# ## Plotting the Distribution of Train and Test Output Data 
plt.style.use(['science','notebook','grid'])

fig, ax = plt.subplots(2,2, figsize=(18,14))

minSand = np.min(y_sand)
maxSand = np.max(y_sand)

binsSand = np.linspace(minSand, maxSand, 8)
# density =True : used to normalise bin heights to make the integral of  histogram 1.
ax[0][0].hist([ySand_train, ySand_test], binsSand , label=['Train', 'Test'], density=True, color = ['blue','red'])
# results in error when yN_train/ yN_test is data frame or ndarray
ax[0][0].legend(loc='upper left', fontsize =12)
ax[0][0].set_xlabel('Sand content',fontsize =16)
ax[0][0].set_ylabel('Normalised frequency',fontsize =12)
ax[0][0].tick_params(axis='both', labelsize=8)

minSilt = np.min(y_silt)
maxSilt = np.max(y_silt)

binsSilt = np.linspace(minSilt, maxSilt, 8)
# density =True : used to normalise bin heights to make the integral of  histogram 1.
ax[0][1].hist([ySilt_train, ySilt_test], binsSilt , label=['Train', 'Test'], density=True, color = ['blue','red'])
# results in error when yN_train/ yN_test is data frame or ndarray
ax[0][1].legend(loc='upper right', fontsize =12)
ax[0][1].set_xlabel('Silt content',fontsize =16)
ax[0][1].set_ylabel('Normalised frequency',fontsize =12)
ax[0][1].tick_params(axis='both', labelsize=8)

minClay = np.min(y_clay)
maxClay = np.max(y_clay)

binsClay = np.linspace(minClay, maxClay, 8)
# density =True : used to normalise bin heights to make the integral of  histogram 1.
ax[1][1].hist([yClay_train, yClay_test], binsClay , label=['Train', 'Test'], density=True, color = ['blue','red'])
# results in error when yN_train/ yN_test is data frame or ndarray
ax[1][1].legend(loc='upper right', fontsize =12)
ax[1][1].set_xlabel('Clay content',fontsize =16)
ax[1][1].set_ylabel('Normalised frequency',fontsize =12)
ax[1][1].tick_params(axis='both', labelsize=8)

minTOC = np.min(y_toc)
maxTOC = np.max(y_toc)

binsTOC = np.linspace(minTOC, maxTOC, 8)
# density =True : used to normalise bin heights to make the integral of  histogram 1.
ax[1][0].hist([yTOC_train, yTOC_test], binsTOC , label=['Train', 'Test'], density=True, color = ['blue','red'])
# results in error when yN_train/ yN_test is data frame or ndarray
ax[1][0].legend(loc='upper right', fontsize =12)
ax[1][0].set_xlabel('Total Organic Content',fontsize =16)
ax[1][0].set_ylabel('Normalised frequency',fontsize =12)
ax[1][0].tick_params(axis='both', labelsize=8)

fig.suptitle('Train Test Distribution of Data', x = 0.5 ,y = .95, fontsize = 20)

#plt.show()

# Finding best Model Parameters (window, n_components)

In [ ]:
# z= y_silt.sort_values().reset_index(drop=True)
# #z= z.reset_index(drop=True)
# len(z)
# z[280]


In [ ]:
from sklearn import linear_model

def best_param_PLSR (spectra, target, rand_st, tst_siz, window, n_comp):
    X= spectra.copy()
    y= target.copy()
     
    iqrpM = np.zeros(shape=(window, n_comp))
    # test train split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = tst_siz, random_state=rand_st)
    for w in range (0,window):
        X_train_smth = sgsmooth (X_train, w)
        X_test_smth = sgsmooth (X_test, w)
        for n in range(1,n_comp):
            Model = PLSRegression(n_components=n, scale=True)
            
            Model.fit(X_train_smth, y_train)
            y_pred = Model.predict(X_test, copy=True)
            yhat_pred = Model.predict(X_train, copy=True)            
            
            iqrp_test = iqrp(y_pred, y_test)
            iqrp_train = iqrp(yhat_pred, y_train)
            
            iqrpM[w,n] = iqrp_test
            
            if iqrp_test == iqrpM.max():
                bestw = w
                bestn = n
                maxIQRP = iqrp_test
                print('Found new best at w=', w, 'n=', n,  '>>   IQRP_test =', iqrp_test, '>>   IQRP_train =', iqrp_train)
                
    return (bestw, bestn, maxIQRP)


# spectra = udf.iloc[:, 7:2158].copy()
# w1, n1, IQRP1 = best_param_PLSR (spectra, y_sand, rand_sand, 0.2, 3,20) 

# spectra = udf.iloc[:, 7:2158].copy()
# w2, n2, IQRP2 = best_param_PLSR (spectra, y_silt, rand_silt, 0.2, 3,20)  

# spectra = udf.iloc[:, 7:2158].copy()
# w3, n3, IQRP3 = best_param_PLSR (spectra, y_clay, rand_clay, 0.2, 3,20)

# spectra = udf.iloc[:, 7:2158].copy()
# w4, n4, IQRP4 = best_param_PLSR (spectra, y_toc, rand_toc, 0.2, 3, 20)


In [ ]:
#print('Sand:', w1, n1, IQRP1, 'Silt:', w2, n2, IQRP2, 'Clay:', w3, n3, IQRP3, 'TOC', w4, n4, IQRP4) 

In [ ]:
spectra = udf.iloc[:, 7:2158].copy()
w1, n1, IQRP1 = best_param_PLSR (spectra, y_sand, rand_sand, 0.2, 3,20)

In [ ]:
spectra = udf.iloc[:, 7:2158].copy()
w2, n2, IQRP2 = best_param_PLSR (spectra, y_silt, rand_silt, 0.2, 3,20)  

In [ ]:
spectra = udf.iloc[:, 7:2158].copy()
w3, n3, IQRP3 = best_param_PLSR (spectra, y_clay, rand_clay, 0.2, 3,20)

In [ ]:
spectra = udf.iloc[:, 7:2158].copy()
w4, n4, IQRP4 = best_param_PLSR (spectra, y_toc, rand_toc, 0.2, 3, 20)

# Step 4: Training the Model

        - Create a linear regr object
        - Apply fit function (using regr object) on training data
        - Check the weights/parameters generated by fit function 

In [ ]:
# spectra = udf.iloc[:,7:2158].copy()
# smth_spec = sgsmooth (spectra,50)

# smth_spec.iloc[1,:].plot()
# plt.show()
# spectra.iloc[1,:].plot()
# plt.show()

In [ ]:
from sklearn import linear_model

regSand = PLSRegression(n_components=n1, scale=True)
regSilt = PLSRegression(n_components=n2, scale=True)
regClay = PLSRegression(n_components=n3, scale=True)
regTOC = PLSRegression(n_components=n4, scale=True)


# train the model using X_train_smth and y_train (identified in previous step)

XSand_train_smth = sgsmooth (XSand_train, w1)       
regSand.fit(XSand_train_smth, ySand_train)

XSilt_train_smth = sgsmooth (XSilt_train, w2)
regSilt.fit(XSilt_train_smth, ySilt_train)

XClay_train_smth = sgsmooth (XClay_train, w3)
regClay.fit(XClay_train_smth, yClay_train)

XTOC_train_smth = sgsmooth (XTOC_train, w4)
regTOC.fit(XTOC_train_smth, yTOC_train)



# Step 5: Prediction on Test Data 

In [ ]:
# Predition using the trained-Model on the "Test Data"

ySand_pred = regSand.predict(XSand_test,copy=True)
ySand_pred = np.round(ySand_pred, 2)

ySilt_pred = regSilt.predict(XSilt_test,copy=True)
ySilt_pred = np.round(ySilt_pred, 2)

yClay_pred = regClay.predict(XClay_test,copy=True)
yClay_pred = np.round(yClay_pred, 2)

yTOC_pred = regTOC.predict(XTOC_test,copy=True)
yTOC_pred = np.round(yTOC_pred, 2)


# Step 6: Model Accuracy and Plots

In [ ]:
# Merging actual and predicted (Test data) in one data frame for plots
ySand_tp = pd.DataFrame({'actual':ySand_test.values, 'predic': ySand_pred[:,0]})
# ySand_sortedA = ySand_tp.sort_values(by =['actual'])
# ySand_srtA = ySand_sortedA.reset_index(drop=True)

ySilt_tp = pd.DataFrame({'actual':ySilt_test.values, 'predic': ySilt_pred[:,0]})
# ySilt_sortedA = ySilt_tp.sort_values(by =['actual'])
# ySilt_srtA = ySilt_sortedA.reset_index(drop=True)

yClay_tp = pd.DataFrame({'actual':yClay_test.values, 'predic': yClay_pred[:,0]})
# yClay_sortedA = yClay_tp.sort_values(by =['actual'])
# yClay_srtA = yClay_sortedA.reset_index(drop=True)

yTOC_tp = pd.DataFrame({'actual':yTOC_test.values, 'predic': yTOC_pred[:,0]})
# yTOC_sortedA = yTOC_tp.sort_values(by =['actual'])
# yTOC_srtA = yTOC_sortedA.reset_index(drop=True)

In [ ]:
# # Plots for Visualising the Model Accuracy
#plt.style.use(['science','notebook','grid'])

def plot_pred_acc (target):

    fig, axes = plt.subplots(1,1, figsize=(8,7))

    if target == 'sand':
        z = np.polyfit(ySand_test, ySand_pred, 1)
        score_cv = r2_score(ySand_test, ySand_pred)
        with plt.style.context(('ggplot')):
            ySand_tp.plot.scatter(ax= axes, x="actual", y="predic", alpha=0.8, color = balu, edgecolors='k')
            axes.plot(np.polyval(z, ySand_test), ySand_test, c='blue', linewidth=1)
            axes.plot(ySand_test, ySand_test, color='green', linewidth=1)
            axes.tick_params(axis='both', labelsize=10)
            plt.text(1, 11, 'SAND', fontsize = 20, color = balu)
            plt.text(1, 10.2, 'IQRP ={:.2f}'.format(IQRP1), fontsize = 16)
            plt.text(1, 9.4, 'R2 ={:.2f}'.format(np.round(score_cv,3)), fontsize = 16)

    elif target == 'clay':
        z = np.polyfit(yClay_test, yClay_pred, 1)
        score_cv = r2_score(yClay_test, yClay_pred)
        with plt.style.context(('ggplot')):
            yClay_tp.plot.scatter(ax= axes, x="actual", y="predic", alpha=0.8, color = mati, edgecolors='k')
            axes.plot(np.polyval(z, yClay_test), yClay_test, c='blue', linewidth=1)
            axes.plot(yClay_test, yClay_test, color='green', linewidth=1)
            axes.tick_params(axis='both', labelsize=10)
            plt.text(1, 11, 'CLAY', fontsize = 20, color = mati)
            plt.text(1, 10.2, 'IQRP ={:.2f}'.format(IQRP3), fontsize = 16)
            plt.text(1, 9.4, 'R2 ={:.2f}'.format(np.round(score_cv,3)), fontsize = 16)

        
    elif target == 'silt':
        z = np.polyfit(ySilt_test, ySilt_pred, 1)
        score_cv = r2_score(ySilt_test, ySilt_pred)
        with plt.style.context(('ggplot')):
            ySilt_tp.plot.scatter(ax= axes, x="actual", y="predic", alpha=0.8, color = kado, edgecolors='k')
            axes.plot(np.polyval(z, ySilt_test), ySilt_test, c='blue', linewidth=1)
            axes.plot(ySilt_test, ySilt_test, color='green', linewidth=1)
            axes.tick_params(axis='both', labelsize=10)
            plt.text(1, 11, 'SILT', fontsize = 20, color = kado)
            plt.text(1, 10.2, 'IQRP ={:.2f}'.format(IQRP2), fontsize = 16)
            plt.text(1, 9.4, 'R2 ={:.2f}'.format(np.round(score_cv,3)), fontsize = 16)


    elif target == 'TOC':
        z = np.polyfit(yTOC_test, yTOC_pred, 1)
        score_cv = r2_score(yTOC_test, yTOC_pred)
        with plt.style.context(('ggplot')):
            yTOC_tp.plot.scatter(ax= axes, x="actual", y="predic", alpha=0.8, color = 'green', edgecolors='k' )
            axes.plot(np.polyval(z, yTOC_test), yTOC_test, c='blue', linewidth=1)
            axes.plot(yTOC_test, yTOC_test, color='green', linewidth=1)
            axes.tick_params(axis='both', labelsize=10)
            plt.text(1, 11, 'TOC', fontsize = 20, color = 'green')
            plt.text(1, 10.2, 'IQRP ={:.2f}'.format(IQRP4), fontsize = 16)
            plt.text(1, 9.4, 'R2 ={:.2f}'.format(np.round(score_cv,3)), fontsize = 16)

        
    else:  
        ySand_srtA.plot.scatter(ax= axes, x="actual", y="predic", alpha=0.8, color = balu)
        axes.tick_params(axis='both', labelsize=10)
        plt.text(3, 11, 'SAND', fontsize = 20, color = balu)
        yClay_srtA.plot.scatter(ax= axes, x="actual", y="predic", alpha=0.8, color = mati)
        axes.tick_params(axis='both', labelsize=10)
        plt.text(3, 10, 'CLAY', fontsize = 20, color = mati)
        ySilt_srtA.plot.scatter(ax= axes, x="actual", y="predic", alpha=0.8, color = kado)
        axes.tick_params(axis='both', labelsize=10)
        plt.text(1, 11, 'SILT', fontsize = 20, color = kado)
        yTOC_srtA.plot.scatter(ax= axes, x="actual", y="predic", alpha=0.8, color = 'green')
        axes.tick_params(axis='both', labelsize=10)
        plt.text(1, 10, 'TOC', fontsize = 20, color = 'green')
     

    fig.suptitle('Predictions on "Test" Data', x = 0.5 ,y = .95, fontsize=20)
    plt.xlim([0, 12])
    plt.ylim([0, 12])
    plt.show()

ipywidgets.interact(plot_pred_acc, target = ['sand', 'clay', 'silt', 'TOC', 'All'])

